Example of how to solve a single non-linear equation, from HW3, problem 1, first part of part c

Start, I imported some libraries I'll need later. Then, I made a function with the equation(s) to solve. Here, there is just one equation. The return could be an array of equations. The first argument (here, x_out) *must* be the variable we want to determine (the dependent variable). Here, it is a scalar. If we have a system of equations, it would be an array.

In [1]:
import numpy as np
from scipy.optimize import fsolve

def pfr_design_eq(x_out, x_in, vol, nuo, k):
    """
    PFR design eq for HW3 problem 1, set up for f(Xi) = 0 for fsolve function
    :param x_in: initial conversion (unitless)
    :param x_out: final conversion (unitless)
    :param vol: PFR volume in L
    :param nuo: volumetric flow in L/min
    :param k: rate coefficient in 1/min
    :return: function residual (want close to zero)
    """
    return vol - nuo / k * (4.0 * np.log(1 / (1 - x_out)) - 3.0 * x_out - 4.0 * np.log(1 / (1 - x_in)) + 3.0 * x_in)

Let's also define a function to determine k at a new temperature

In [2]:
def k_at_new_temp(k_ref, e_a, r_gas, t_ref, t_new):
    """
    convert using "alternate" form of Arrhenius eq
    :param k_ref: reference rate coefficient at temp t_ref
    :param e_a: activation energy with units consistent with given r_gas
    :param r_gas: universal gas constant in units consistent with e_a and temps
    :param t_ref: reference temp in K
    :param t_new: new temp in K
    :return: k at the "new' temperature, t_ref, in the same units as the given k_ref
    """
    return k_ref * np.exp((-e_a/r_gas)*(1/t_new - 1/t_ref))

And one to convert temperatures, as well as to determine k at a new temp from k at an old temp

In [3]:
def temp_c_to_k(temp_in_C):
    """
    simple conversion
    :param temp_in_C:
    :return: temp in K
    """
    return temp_in_C + 273.15

def k_at_new_temp(k_ref, e_a, r_gas, t_ref, t_new):
    """
    convert using "alternate" form of Arrhenius eq
    :param k_ref: reference rate coefficient at temp t_ref
    :param e_a: activation energy with units consistent with given r_gas
    :param r_gas: universal gas constant in units consistent with e_a and temps
    :param t_ref: reference temp in K
    :param t_new: new temp in K
    :return: k at the "new' temperature, t_ref, in the same units as the given k_ref
    """
    return k_ref * np.exp((-e_a/r_gas)*(1/t_new - 1/t_ref))


Now, let's define some constants and our given data, and then call the solver.

In [4]:
R_J = 8.314472  # universal gas constant in J / K mol
R_ATM = 0.082057338  # universal gas constant in atm-L / mol-K
 
t_ref = temp_c_to_k(150)  # reference temp, converting Celsius to K
temp = temp_c_to_k(227)  # temp for problem in K
e_a = 85.0  # in kJ/mol
k_ref = 5e-3  # min^-1, at 150 C
k_new = k_at_new_temp(k_ref, e_a, R_J / 1000.0, t_ref, temp)
nuo = 2.5 / 10 * R_ATM * temp  # volumetric flow in L/min
volume_pfr_b = 323.93 # L

x_in = 0.0
x_out_guess = 0.5
new_pfr_x_out = fsolve(pfr_design_eq, 0.75, args=(x_in, volume_pfr_b / 2.0, nuo, k_new))[0]
print("PFR outlet conversion: {:.2f}".format(new_pfr_x_out))


PFR outlet conversion: 0.75
